In [1]:
import numpy as np
#import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import autograd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transform_test = transforms.Compose([
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
batch_size = 128
testset = torchvision.datasets.CIFAR10(root='./', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)
testloader = enumerate(testloader)

cuda:0


In [8]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()               # input dim
        self.conv1 = self.conv(3  ,196, 3,1,1, (196,32,32)) # 32*32
        self.conv2 = self.conv(196,196, 3,2,1, (196,16,16)) # 32*32
        self.conv3 = self.conv(196,196, 3,1,1, (196,16,16)) # 16*16
        self.conv4 = self.conv(196,196, 3,2,1, (196,8,8))   # 16*16
        self.conv5 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv6 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv7 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv8 = self.conv(196,196, 3,2,1, (196,4,4))   # 8*8
        self.pool = nn.MaxPool2d(4,4)           # 4*4
        
        self.fc1 = nn.Linear(196, 1)            # 1     critic output
        self.fc10 = nn.Linear(196, 10)          # 10    auxiliary classifier output
        
    def conv(self,i,o,k,s,p,ln_dim):
        return nn.Sequential(
            nn.Conv2d(i,o,k,s,p),
            nn.LayerNorm(ln_dim),
            nn.LeakyReLU()
        )
        
    def forward(self, x, extract_features=8):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if(extract_features==3):
            h = F.max_pool2d(x,16,16)
            h = h.view(-1, 196)
            return h
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        if(extract_features==6):
            h = F.max_pool2d(x,8,8)
            h = h.view(-1, 196)
            return h
        x = self.conv7(x)
        x = self.conv8(x)
        if(extract_features==8):
            h = F.max_pool2d(x,4,4)
            h = h.view(-1, 196)
            return h
        x = self.pool(x)
        x1 = x.view(-1, 196)
        critic = self.fc1(x1)
        x2 = x.view(-1, 196)
        classifier = self.fc10(x2)
        return critic,classifier




In [9]:

model = torch.load('discriminator.model')
model.to(device)
model.eval()

C:\Users\wangwang\Anaconda3\lib\site-packages\torch\serialization.py:391: UserWarning: Couldn't retrieve source code for container of type discriminator. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


discriminator(
  (conv1): Sequential(
    (0): Conv2d(3, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LayerNorm(torch.Size([196, 32, 32]), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv2): Sequential(
    (0): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv3): Sequential(
    (0): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv4): Sequential(
    (0): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LayerNorm(torch.Size([196, 8, 8]), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv5): Sequential(
    (0): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), p

In [10]:
batch_idx, (X_batch, Y_batch) = testloader.__next__()
X_batch = Variable(X_batch,requires_grad=True).to(device)
Y_batch = Variable(Y_batch).to(device)

X = X_batch.mean(dim=0)
X = X.repeat(10,1,1,1)

Y = torch.arange(10).type(torch.int64)
Y = Variable(Y).to(device)

In [11]:
def plot(samples):
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(10, 10)
    gs.update(wspace=0.02, hspace=0.02)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample)
    return fig

X = X_batch.mean(dim=0)
X = X.repeat(batch_size,1,1,1)

Y = torch.arange(batch_size).type(torch.int64)
Y = Variable(Y).cuda()

lr = 0.1
weight_decay = 0.001
for i in range(200):
    output = model(X)

    loss = -output[torch.arange(batch_size).type(torch.int64),torch.arange(batch_size).type(torch.int64)]
    gradients = torch.autograd.grad(outputs=loss, inputs=X,
                              grad_outputs=torch.ones(loss.size()).cuda(),
                              create_graph=True, retain_graph=False, only_inputs=True)[0]

    prediction = output.data.max(1)[1] # first column has actual prob.
    accuracy = ( float( prediction.eq(Y.data).sum() ) /float(batch_size))*100.0
    print(i,accuracy,-loss)

    X = X - lr*gradients.data - weight_decay*X.data*torch.abs(X.data)
    X[X>1.0] = 1.0
    X[X<-1.0] = -1.0

## save new images
samples = X.data.cpu().numpy()
samples += 1.0
samples /= 2.0
samples = samples.transpose(0,2,3,1)

fig = plot(samples[0:100])
plt.savefig('visualization/max_features_8_gen.png', bbox_inches='tight')
plt.close(fig)

0 0.78125 tensor([ 0.7270,  1.5844,  6.5330,  3.8343,  5.1098,  1.1005,  0.4101,  3.7226,
         1.0784,  0.9281,  0.9583,  1.2532,  0.5053,  1.2971, -0.0009,  0.7618,
         1.1319,  1.8214,  1.8187,  0.1582,  2.1162,  1.9057,  1.8027,  2.9358,
         0.9970, -0.0019,  1.4472, -0.0021,  1.3615,  1.0343,  1.5283,  3.0530,
         1.9065,  2.3509, -0.0022,  0.0850,  1.2895,  4.0620,  1.3591,  1.9234,
         0.0646,  0.6199,  1.0001,  1.2422,  0.8210,  1.2922,  1.2607,  1.4007,
         1.6234,  2.6137, -0.0005,  2.0292,  3.1549,  2.1950,  1.0746,  0.9726,
         1.8948,  1.2921,  0.9504,  2.0920,  0.6291,  1.6277,  2.4525,  1.5324,
         3.1974,  1.3394,  0.8135,  2.6093,  1.2078,  2.5907,  0.4333,  1.7670,
         2.6426,  0.9173,  1.2863,  1.5444,  1.3804,  1.1276,  0.7480,  0.3974,
         0.7917,  0.7271,  1.6345, -0.0033,  2.2168,  1.0756,  0.7679,  0.8389,
         2.0066,  1.6106,  0.9272,  0.8973,  0.0235,  0.1111,  0.4146,  0.3861,
         0.9451,  1.0569,  1.6

9 20.3125 tensor([ 0.9441,  5.5229,  9.8155,  7.5102, 11.3593,  1.9021,  4.4319, 11.6672,
         1.8280,  2.1583,  2.0409,  1.9160,  0.8904,  1.5375, -0.0008,  1.5622,
         1.4691,  9.3203,  5.7857,  1.1341,  9.0018, 10.1332,  3.2611,  5.4016,
         1.3023, -0.0018,  2.1228, -0.0021,  1.9713,  1.6881,  2.1354,  3.8984,
         9.7750,  5.6002, -0.0022,  0.4812,  5.6894,  6.8058,  1.8582,  4.3853,
         0.4588,  1.1551,  1.4792,  1.8502,  1.2013,  1.5395,  3.9164,  1.9132,
         2.3619,  4.8498, -0.0004,  2.7068,  7.0705,  3.1914,  1.4918,  1.3782,
         2.6841,  7.8905,  1.2641,  7.7445,  1.2912,  2.2506,  3.0932,  7.0861,
        12.1876,  2.3139, 10.5040,  4.0688,  2.7340,  8.8552,  1.5674,  2.2405,
         3.1519,  5.0939, 11.1935,  1.8641,  1.8961,  4.8770,  1.5751,  5.5754,
         3.2118,  1.1598,  3.7908, -0.0033,  2.8545,  1.6736,  1.2572,  1.2363,
         2.9328,  9.6445,  1.3001,  6.4575,  0.1078,  0.7810,  0.8605,  0.7572,
         1.2111,  1.4901,  2.2

18 26.5625 tensor([ 1.0634,  7.4264, 11.5085,  9.5316, 14.0721,  2.4721,  7.4020, 14.4688,
         2.4184,  2.8866,  2.7659,  2.3502,  1.2352,  1.6961, -0.0008,  2.0139,
         1.7177, 13.2623,  8.5627,  1.9857, 12.1408, 14.2989,  4.3320,  6.3512,
         1.5412, -0.0018,  2.5413, -0.0021,  2.4310,  2.1926,  2.5878,  4.2785,
        13.5199,  7.4997, -0.0021,  0.7238,  8.0742,  8.3179,  2.2156,  6.0143,
         0.7865,  1.5977,  1.8669,  2.5431,  1.4698,  1.7256,  5.8750,  2.1708,
         2.9601,  6.3802, -0.0004,  3.3943,  9.7254,  3.8011,  1.8701,  1.6177,
         3.2689, 10.1561,  1.5676, 10.9584,  1.7684,  2.7155,  3.7085,  9.7876,
        14.3488,  3.0323, 14.4719,  4.7754,  3.9705, 12.5167,  2.7194,  2.7154,
         3.4380,  8.1243, 14.3952,  2.1974,  2.2627,  7.1664,  2.3818,  9.7251,
         4.8235,  1.5674,  5.1188, -0.0033,  3.2409,  2.0902,  1.5791,  1.5573,
         3.4993, 14.0209,  1.5706,  9.3153,  0.1725,  1.2824,  1.1768,  1.0311,
         1.4467,  1.7726,  2.

27 28.90625 tensor([ 1.1820,  8.5792, 12.4275, 11.1030, 15.6727,  2.9400,  9.0805, 15.9054,
         2.8232,  3.2804,  3.3706,  2.7449,  1.4897,  1.8372, -0.0007,  2.3255,
         1.9072, 15.8145, 10.6315,  2.9204, 14.1201, 16.8798,  5.0918,  6.9178,
         1.8491, -0.0018,  2.8947, -0.0021,  2.7161,  2.6880,  2.9501,  4.6443,
        15.5076,  8.7062, -0.0021,  0.9492,  9.6778,  9.5604,  2.5630,  6.9784,
         1.1336,  2.0789,  2.1639,  3.0560,  1.6497,  1.8482,  7.3459,  2.3771,
         3.6234,  7.2934, -0.0003,  4.0889, 11.2714,  4.1790,  2.1422,  1.8101,
         3.7016, 11.8706,  1.8108, 13.1436,  2.1786,  3.0802,  4.0494, 11.6488,
        16.0361,  3.5555, 17.0622,  5.2460,  4.8092, 14.9028,  3.5204,  3.0329,
         3.7326, 10.1372, 16.3565,  2.4454,  2.5164,  8.8863,  2.8659, 12.4490,
         6.1286,  1.9646,  6.1801, -0.0033,  3.6448,  2.4462,  1.8762,  1.8649,
         3.9230, 16.0940,  1.7887, 11.1774,  0.2241,  1.7047,  1.4197,  1.2695,
         1.6337,  1.9998,  2

36 29.6875 tensor([ 1.2966,  9.5493, 13.0053, 12.0128, 16.7542,  3.3064, 10.2785, 16.8980,
         3.1616,  3.6394,  3.9413,  3.0891,  1.7103,  1.9695, -0.0007,  2.5666,
         2.0542, 17.8771, 12.1594,  3.5526, 15.2624, 18.5049,  5.6829,  7.3582,
         2.1053, -0.0017,  3.2381, -0.0021,  2.9337,  3.1978,  3.2487,  5.0866,
        16.7912,  9.5727, -0.0021,  1.1756, 10.9399, 10.6324,  2.9280,  7.8242,
         1.4268,  2.6302,  2.3898,  3.4638,  1.8221,  1.9609,  8.5329,  2.5723,
         4.1555,  8.0729, -0.0002,  4.6761, 12.4524,  4.4959,  2.3571,  2.0208,
         4.0972, 13.3403,  2.0128, 14.8717,  2.4855,  3.3548,  4.2931, 13.0603,
        17.2840,  4.0445, 18.9620,  5.5749,  5.4935, 16.3552,  4.1596,  3.2878,
         3.9877, 11.4100, 17.8110,  2.6687,  2.7507, 10.3642,  3.2578, 14.1863,
         7.1038,  2.2623,  6.9844, -0.0033,  4.0197,  2.6864,  2.1168,  2.1912,
         4.2579, 17.2369,  1.9645, 12.6112,  0.2760,  2.0714,  1.6341,  1.4932,
         1.8040,  2.1933,  3.

...

159 56.25 tensor([ 3.1431, 16.2201, 16.7090, 17.2854, 22.6243,  5.5956, 18.4689, 27.6997,
         7.0077,  8.6047,  8.7341,  6.2705,  3.9368,  3.3207, -0.0002,  4.7912,
         3.5881, 27.8793, 19.9057,  8.5822, 19.9103, 25.9897,  9.3781, 10.8131,
         3.5841, -0.0013,  7.8288, -0.0019,  4.9157,  6.0693,  5.1961,  7.9546,
        24.5702, 14.8306, -0.0015,  4.6455, 18.5706, 16.5010,  5.4777, 15.2572,
         3.2343,  7.1533,  5.2899,  7.5138,  4.0668,  4.2449, 16.7601,  4.4330,
         7.5817, 12.4163,  1.8741,  9.1179, 20.7329,  8.9528,  3.8044,  5.1501,
         7.9738, 21.5008,  3.7523, 22.7251,  6.1197,  5.2532,  5.8601, 20.1095,
        22.6466,  8.2180, 27.6785, 10.5535, 12.6985, 23.4996, 12.2579,  5.3055,
         7.2431, 16.4282, 26.0116,  4.0053,  4.6330, 17.4824,  7.5810, 22.1906,
        13.0857,  3.9594, 11.7450, -0.0030,  7.1876,  4.2386,  6.6980,  5.4871,
         6.6787, 23.1504,  3.3039, 21.1931,  0.7132,  4.5826,  4.2128,  2.9728,
         3.5196,  4.5074,  6.6

168 57.03125 tensor([ 3.2682, 16.4834, 16.8584, 17.4798, 22.8615,  5.7113, 18.8797, 28.1980,
         7.2449,  9.0480,  8.9604,  6.5034,  4.0479,  3.4185, -0.0002,  4.8966,
         3.6839, 28.1090, 20.2534,  8.7993, 20.0586, 26.2296,  9.5473, 11.0383,
         3.6275, -0.0013,  8.0209, -0.0019,  5.0184,  6.1811,  5.3053,  8.1084,
        24.8821, 15.1021, -0.0015,  5.0304, 18.8080, 16.7877,  5.5535, 15.7149,
         3.3035,  7.2899,  5.5209,  7.6921,  4.1824,  4.4396, 17.1194,  4.5290,
         7.7595, 12.5506,  2.0012,  9.3467, 21.1379,  9.2906,  3.8740,  5.3426,
         8.2427, 21.7501,  3.9292, 22.9448,  6.3586,  5.3599,  5.9192, 20.3020,
        22.9441,  8.5891, 27.9877, 10.9394, 13.0270, 23.7386, 12.6008,  5.4328,
         7.4059, 16.5871, 26.3602,  4.0765,  4.7226, 17.7732,  7.7772, 22.5290,
        13.3725,  4.0292, 12.0468, -0.0030,  7.3623,  4.3120,  6.8818,  5.6830,
         6.8332, 23.3685,  3.4057, 21.5663,  0.7337,  4.7054,  4.3940,  3.0349,
         3.6483,  4.6254,  

177 57.03125 tensor([ 3.3832, 16.7408, 17.0035, 17.6611, 23.0889,  5.8264, 19.2331, 28.6396,
         7.4697,  9.4640,  9.1877,  6.7276,  4.1605,  3.5210, -0.0001,  5.0046,
         3.7796, 28.3240, 20.5609,  9.0001, 20.2021, 26.4579,  9.7354, 11.2511,
         3.6693, -0.0013,  8.2005, -0.0019,  5.1154,  6.2793,  5.4122,  8.2591,
        25.1491, 15.3615, -0.0015,  5.3602, 19.0380, 17.0752,  5.6316, 16.1840,
         3.3726,  7.4098,  5.7679,  7.8647,  4.3070,  4.6551, 17.4781,  4.6262,
         7.9417, 12.6727,  2.1378,  9.5797, 21.5072,  9.6233,  3.9426,  5.5318,
         8.5011, 21.9925,  4.1179, 23.1523,  6.5869,  5.4546,  5.9783, 20.4839,
        23.2213,  8.9577, 28.2764, 11.3862, 13.3369, 23.9751, 12.9411,  5.5520,
         7.5447, 16.7349, 26.7090,  4.1398,  4.8215, 18.0781,  7.9710, 22.8608,
        13.6404,  4.0965, 12.3324, -0.0030,  7.5332,  4.3834,  7.0531,  5.8542,
         6.9858, 23.5577,  3.5123, 21.9174,  0.7551,  4.8256,  4.5642,  3.0986,
         3.7602,  4.7340,  

186 56.25 tensor([ 3.4956, 16.9906, 17.1569, 17.8385, 23.3025,  5.9476, 19.5422, 29.0899,
         7.6747,  9.8908,  9.4064,  6.9389,  4.2626,  3.6126, -0.0001,  5.1243,
         3.8729, 28.5318, 20.8491,  9.1883, 20.3378, 26.6785,  9.9176, 11.4382,
         3.7085, -0.0012,  8.3672, -0.0019,  5.2079,  6.3806,  5.5138,  8.4172,
        25.3958, 15.6192, -0.0014,  5.6847, 19.2581, 17.3254,  5.7118, 16.5921,
         3.4465,  7.5249,  6.0180,  8.0183,  4.4348,  4.8558, 17.8280,  4.7219,
         8.1129, 12.7931,  2.2878,  9.8204, 21.8228,  9.9578,  4.0086,  5.7083,
         8.7316, 22.2319,  4.3092, 23.3484,  6.8166,  5.5489,  6.0363, 20.6629,
        23.4771,  9.3116, 28.5412, 11.8686, 13.6349, 24.1826, 13.2544,  5.6557,
         7.6789, 16.8792, 27.0539,  4.2023,  4.9167, 18.3885,  8.1638, 23.2086,
        13.9131,  4.1624, 12.6115, -0.0029,  7.6842,  4.4512,  7.2198,  6.0086,
         7.1601, 23.7342,  3.6233, 22.2298,  0.7765,  4.9474,  4.7078,  3.1644,
         3.8676,  4.8443,  7.1

195 58.59375 tensor([ 3.6189, 17.2107, 17.3081, 18.0100, 23.5157,  6.0832, 19.8226, 29.5122,
         7.8729, 10.2961,  9.6200,  7.1266,  4.3628,  3.7038, -0.0001,  5.2522,
         3.9666, 28.7460, 21.1128,  9.3637, 20.4687, 26.8924, 10.0927, 11.6169,
         3.7489, -0.0012,  8.5234, -0.0019,  5.2875,  6.4933,  5.6050,  8.5732,
        25.6218, 15.8614, -0.0014,  5.9604, 19.4750, 17.5495,  5.7945, 16.9770,
         3.5253,  7.6307,  6.2685,  8.1571,  4.5628,  5.0210, 18.1505,  4.8110,
         8.2740, 12.9121,  2.4460, 10.0501, 22.1024, 10.2934,  4.0766,  5.8914,
         8.9633, 22.4634,  4.4906, 23.5386,  7.0466,  5.6360,  6.0914, 20.8332,
        23.7162,  9.6405, 28.8046, 12.3276, 13.9261, 24.3703, 13.5245,  5.7522,
         7.8108, 17.0227, 27.4005,  4.2627,  5.0039, 18.6902,  8.3610, 23.5470,
        14.1666,  4.2246, 12.9075, -0.0029,  7.8250,  4.5134,  7.3843,  6.1414,
         7.3387, 23.8899,  3.7352, 22.4909,  0.7965,  5.0737,  4.8585,  3.2344,
         3.9635,  4.9429,  